In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import seaborn as sns

%matplotlib inline
tqdm.pandas()

# 1. Prepare dataset

- `bg_adi`: 2020-adjusted ADI percentile for every BGs
- `bg20_adi`: Neighborhood Atlas's newly published 2020 ADI
- `food`: for every BG, show every pantries within 25 mile (-> should be 20 mile)

In [3]:
bg_adi = pd.read_csv('../data/convert_20BG_to_10BG_with_adi.csv', 
                       header=0, sep=',', 
                      dtype={'bg_fips':str, 'ADI_NATRANK':float, 'ADI_STATERNK':float})

bg_adi['ADI_NATRANK'] = np.floor(bg_adi['ADI_NATRANK'])
bg_adi['ADI_STATERNK'] = np.floor(bg_adi['ADI_STATERNK'])

In [5]:
bg_adi.head()

,bg_fips,ADI_NATRANK,ADI_STATERNK
0,010010201001,85.0,7.0
1,010010201002,52.0,2.0
2,010010202001,88.0,8.0
3,010010202002,82.0,6.0
4,010010203001,71.0,5.0


In [21]:
bg20_adi = pd.read_csv('../data/US_2020_ADI_Census Block Group_v3.2.csv', 
                        dtype={'FIPS':str, 'ADI_NATRANK':str, 'ADI_STATERNK':str})
bg20_adi = bg20_adi[['FIPS', 'ADI_NATRANK', 'ADI_STATERNK']]
bg20_adi = bg20_adi.rename(columns={'FIPS':'bg_fips'})

In [30]:
bg20_adi

,bg_fips,ADI_NATRANK,ADI_STATERNK
0,010010201001,73,5
1,010010201002,62,3
2,010010202001,83,7
3,010010202002,87,7
4,010010203001,73,5
...,...,...,...
242330,721537506011,92,6
242331,721537506012,87,4
242332,721537506013,93,7
242333,721537506021,98,10


In [14]:
food = pd.read_pickle('../data/food_adi_converted_bg20_with_st_county.pkl')

In [16]:
food.head()

,address,ad_lat,ad_lon,ad_bg_fips,bg_fips,bg_lat,bg_lon,distance_mi,ADI_NATRANK,ADI_STATERNK,bg_state,address_state,bg_county,address_county
0,"80 Richmond Townhouse Rd, Carolina, RI 02812",41.489365,-71.660706,440090506004,090116903001,41.362105,-72.107022,24.720511,54,8,CT,RI,09011,44009
1,"80 Richmond Townhouse Rd, Carolina, RI 02812",41.489365,-71.660706,440090506004,090116904001,41.356744,-72.108543,24.928589,70,9,CT,RI,09011,44009
2,"80 Richmond Townhouse Rd, Carolina, RI 02812",41.489365,-71.660706,440090506004,090116904002,41.351946,-72.106542,24.957116,65,9,CT,RI,09011,44009
3,"80 Richmond Townhouse Rd, Carolina, RI 02812",41.489365,-71.660706,440090506004,090116905003,41.353296,-72.101670,24.688172,69,10,CT,RI,09011,44009
4,"80 Richmond Townhouse Rd, Carolina, RI 02812",41.489365,-71.660706,440090506004,090116905001,41.358894,-72.102416,24.578674,69,10,CT,RI,09011,44009


`food_valid`: we need to import this (which includes all original addresses we collected by computing geo-distances between a food pantry point and neighborhoods within 25 miles) -> Then, we need to merge it with the newly published 2020 ADI valud for the updated database.

In [17]:
food_valid = pd.read_pickle('../data/food_valid_filtered_all_combined.pkl')

In [19]:
food_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37154567 entries, 0 to 37154566
Data columns (total 8 columns):
 #   Column       Dtype  
---  ------       -----  
 0   address      object 
 1   ad_lat       float64
 2   ad_lon       float64
 3   ad_bg_fips   object 
 4   bg_fips      object 
 5   bg_lat       float64
 6   bg_lon       float64
 7   distance_mi  float64
dtypes: float64(5), object(3)
memory usage: 2.2+ GB


## Merge 2020 ADI with `food_valid`

In [20]:
print(len(food_valid['address'].unique()), len(food_valid['bg_fips'].unique()))

34475 238536


In [23]:
food_valid_adi = food_valid.merge(bg20_adi.drop_duplicates(subset=['bg_fips']), how='left', on='bg_fips')

In [24]:
food_valid_adi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37154567 entries, 0 to 37154566
Data columns (total 10 columns):
 #   Column        Dtype  
---  ------        -----  
 0   address       object 
 1   ad_lat        float64
 2   ad_lon        float64
 3   ad_bg_fips    object 
 4   bg_fips       object 
 5   bg_lat        float64
 6   bg_lon        float64
 7   distance_mi   float64
 8   ADI_NATRANK   object 
 9   ADI_STATERNK  object 
dtypes: float64(5), object(5)
memory usage: 3.0+ GB


In [26]:
food_valid_adi_no_nan = food_valid_adi.loc[food_valid_adi['ADI_NATRANK'].isna()==False]
food_valid_adi_no_nan_no_invalid = food_valid_adi_no_nan.loc[food_valid_adi_no_nan['ADI_NATRANK'].str.isnumeric()]

In [31]:
food_valid_adi_no_nan_no_invalid.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 35940827 entries, 0 to 37154566
Data columns (total 10 columns):
 #   Column        Dtype  
---  ------        -----  
 0   address       object 
 1   ad_lat        float64
 2   ad_lon        float64
 3   ad_bg_fips    object 
 4   bg_fips       object 
 5   bg_lat        float64
 6   bg_lon        float64
 7   distance_mi   float64
 8   ADI_NATRANK   int64  
 9   ADI_STATERNK  int64  
dtypes: float64(5), int64(2), object(3)
memory usage: 2.9+ GB


In [28]:
food_valid_adi_no_nan_no_invalid['ADI_NATRANK'] = food_valid_adi_no_nan_no_invalid['ADI_NATRANK'].astype('int')
food_valid_adi_no_nan_no_invalid['ADI_STATERNK'] = food_valid_adi_no_nan_no_invalid['ADI_STATERNK'].astype('int')

/var/folders/qd/3csq0x3s6x56yvjlmpx4svqm0000gn/T/ipykernel_19501/3850011781.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_valid_adi_no_nan_no_invalid['ADI_NATRANK'] = food_valid_adi_no_nan_no_invalid['ADI_NATRANK'].astype('int')
/var/folders/qd/3csq0x3s6x56yvjlmpx4svqm0000gn/T/ipykernel_19501/3850011781.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_valid_adi_no_nan_no_invalid['ADI_STATERNK'] = food_valid_adi_no_nan_no_invalid['ADI_STATERNK'].astype('int')


In [29]:
food_valid_adi_no_nan_no_invalid

,address,ad_lat,ad_lon,ad_bg_fips,bg_fips,bg_lat,bg_lon,distance_mi,ADI_NATRANK,ADI_STATERNK
0,"80 Richmond Townhouse Rd, Carolina, RI 02812",41.489365,-71.660706,440090506004,090116903001,41.362105,-72.107022,24.720511,47,7
1,"80 Richmond Townhouse Rd, Carolina, RI 02812",41.489365,-71.660706,440090506004,090116904001,41.356744,-72.108543,24.928589,82,10
2,"80 Richmond Townhouse Rd, Carolina, RI 02812",41.489365,-71.660706,440090506004,090116904002,41.351946,-72.106542,24.957116,63,9
3,"80 Richmond Townhouse Rd, Carolina, RI 02812",41.489365,-71.660706,440090506004,090116905003,41.353296,-72.101670,24.688172,68,9
4,"80 Richmond Townhouse Rd, Carolina, RI 02812",41.489365,-71.660706,440090506004,090116905001,41.358894,-72.102416,24.578674,73,10
...,...,...,...,...,...,...,...,...,...,...
37154562,"1409 Park Ave, Woonsocket, RI 02895",41.984573,-71.515750,440070175003,440070184002,42.014946,-71.466851,3.270068,53,8
37154563,"1409 Park Ave, Woonsocket, RI 02895",41.984573,-71.515750,440070175003,440070184004,42.008856,-71.470713,2.855163,53,8
37154564,"1409 Park Ave, Woonsocket, RI 02895",41.984573,-71.515750,440070175003,440070185001,41.990746,-71.488182,1.477591,50,7
37154565,"1409 Park Ave, Woonsocket, RI 02895",41.984573,-71.515750,440070175003,440070185002,41.998316,-71.481420,2.001014,55,8


In [32]:
food_valid_adi_no_nan_no_invalid = food_valid_adi_no_nan_no_invalid.reset_index(drop=True)

In [33]:
food_valid_adi_no_nan_no_invalid.to_csv('../data/food_bg20_adi.csv', index=False)

Match with state and county

In [34]:
def find_state_fips(bg, state_data):
    
    st_fips = bg[:2]
    for i in range(len(state_data)):
        if state_data['st_fips'][i] == st_fips:            
            return state_data['st_abb'][i]

state_fips = pd.read_csv('../data/us-state-fips.csv',
                        dtype = {'stname':str, ' st':str, ' stusps':str})
state_fips = state_fips.rename(columns={'stname':'STATE', ' st':'st_fips', ' stusps':'st_abb'})


state_fips.st_abb = state_fips.st_abb.str.strip()
state_fips.st_fips = state_fips.st_fips.str.strip()


In [35]:
food_valid_adi_no_nan_no_invalid['bg_state'] = food_valid_adi_no_nan_no_invalid.progress_apply(lambda x:find_state_fips(x.bg_fips, state_fips), axis=1)
food_valid_adi_no_nan_no_invalid['address_state'] = food_valid_adi_no_nan_no_invalid.progress_apply(lambda x:find_state_fips(x.ad_bg_fips, state_fips), axis=1)

100%|██████████| 35940827/35940827 [2:46:23<00:00, 3600.07it/s]  


In [36]:
#food_valid_adi_no_nan_no_invalid.to_pickle('../data/food_bg20_adi_st.pkl')

In [40]:
len(food_valid_adi_no_nan_no_invalid.bg_state.unique())

51

unique `bg_fips`

In [41]:
len(food_valid_adi_no_nan_no_invalid.bg_fips.unique())

232735

# Data Statistics

- Number of food pantries per state: 34475
- Number of unique BGs: 232,735

In [45]:
num_unique_pantries = len(food_valid_adi_no_nan_no_invalid.address.unique())
print(f"number of FPs in total: {num_unique_pantries}")

num_unique_bgs = len(food_valid_adi_no_nan_no_invalid.bg_fips.unique())
print(f"number of unique BGs: {num_unique_bgs}")

number of FPs in total: 34475
number of unique BGs: 232735


- Number of FPs per state: NY, TX, CA, FL, OH (Top) / DC, DE, WY, HI, SD, AK (Bottom) - from `num_pantries_state`

- `num_served_bgs_state`: the number of BGs that food pantries served per state 

In [47]:
num_pantries_state = food_valid_adi_no_nan_no_invalid.groupby('address').first().address_state.value_counts()
num_served_bgs_state = food_valid_adi_no_nan_no_invalid.address_state.value_counts()

The number of BGs that a food pantry in each state serves

In [50]:
pd.DataFrame(num_served_bgs_state/num_pantries_state).address_state.sort_values(ascending=False)

NY    4595.257560
NJ    4241.696093
DC    3126.292683
CA    2102.337966
MA    1636.473039
MD    1457.955277
IL    1439.653687
CT    1186.041516
RI    1130.811111
PA    1052.225630
MI     987.527959
AZ     985.512121
TX     985.314534
DE     953.141176
NV     929.096939
OH     884.404947
CO     841.107720
WA     838.378750
MN     806.294926
MO     759.318335
GA     758.430355
FL     729.150931
OR     581.739362
IN     551.693878
VA     548.133584
UT     496.280000
KY     473.880503
NC     459.071843
LA     458.334254
NH     428.291667
WI     419.881188
TN     385.777908
HI     364.328000
OK     360.989537
KS     346.024709
SC     341.887946
NE     323.809969
AL     294.473310
NM     259.846154
ID     205.573864
IA     194.691176
MS     193.280000
WV     188.069565
AR     174.538144
ME     165.232719
VT     122.450000
AK     110.506849
SD      82.326389
ND      46.313043
WY      43.682692
MT      41.642276
Name: address_state, dtype: float64